In [12]:
import re
import pandas as pd
import numpy as np
import requests
from lxml import html, etree
import time
import json
import datetime

PROXIES = {
  'http': 'http://VPN4726:@ar.finevpn.org',
  'https': 'https://VPN4726:@ar.finevpn.org',
}

PROXIES ={}

In [15]:
## http://docs.python-guide.org/en/latest/scenarios/scrape/
##

class news_coin_tel_parser():
    def __init__(self):
        self.link = ''
        self.save_dir = "news_cointelegraph_all/"
    
    def read_article_text(self,article_href):
        article_page = requests.get(article_href, proxies=PROXIES)
        article_html = html.fromstring(article_page.content)
        
        try:
            refs = article_html.xpath('//div[@class="referral_widget"]')[0]
            refs_likes = refs.xpath('.//div[@class="referral_stats total-views"]/span[@class="total-qty"]/text()')[0]
            refs_likes = int(refs_likes)

            refs_shares = refs.xpath('.//div[@class="referral_stats total-shares"]/span[@class="total-qty"]/text()')[0]
            refs_shares = int(refs_shares)
        except:
            print ("error in reading likes for ",article_href)
            refs_shares = 0
            refs_likes = 0
        ## print(refs_likes,refs_shares)
        
        try:
            article_text = article_html.xpath('//div[@class="post-full-text contents"]')[0]
            text = etree.tostring(article_text).decode("utf-8") 
        except:
            print ("error reading text from page: ",article_href)
            text = ""
        ##text = re.sub("<figure.*?</figure>","",text)
        ##text = re.findall("<p>.*?</p>",text)
        ##text = "".join([x for x in text])
        return refs_likes,refs_shares,text
    
    def clean_text (self,article_text):
        article_text = re.sub("<.*?>","",article_text)
        article_text = re.sub("&#8217;","'",article_text)
        article_text = re.sub("&#[0-9]+;|\n"," ",article_text)
        article_text = re.sub(" {1,}"," ",article_text)

        return article_text

    def parse_page(self,page_num, verbose = True):
        
        page_link = "https://cointelegraph.com/api/v1/ajax/posts/next"
        data_val = {'page':page_num,'lang':'en'}
        r = requests.post(page_link, data = data_val)
        
        data_dump = json.loads(r.text)
        
        if 'posts' not in data_dump.keys():
            print ("error on requesting page ", page_num)
        
        for i_article in data_dump['posts']:
            
            ## parsing date
            ##
            art_time = i_article['date']
            if len(re.findall('HOURS', art_time))>0 or len(re.findall('MINUTES', art_time))>0:
                art_time = datetime.datetime.now()
            else:
                art_time = datetime.datetime.strptime(art_time, '%b %d, %Y')
            art_time = str(art_time)
            art_date = art_time[0:10]

            art_title = i_article['title']
            art_lead_text = i_article['lead']
            art_href = i_article['url']
            
            if verbose: print (art_date,art_time, art_title,art_href)
        
            refs_likes, refs_shares, article_text = self.read_article_text(art_href)
            
            ## cleaning article from special symbols
            article_text = self.clean_text(article_text)
            
            ## saving article
            fn = re.sub("[^a-zA-Z0-9 ]","",art_title[0:20])
            file_name = self.save_dir + art_date + "_" + 'p_' + '{num:03d}'.format(num=page_num) + "_" + fn + ".txt"
            ##file_name = self.save_dir + art_date + "_"  + fn + ".txt"

            if verbose: print ('FILE NAME: ',file_name)
            f = open(file_name,'w', encoding = 'utf-8')
            f.write(art_title + "\n")
            f.write(art_time + "\n")
            f.write("<brief>" + art_lead_text + "</brief>\n")
            f.write("<likes>" + str(refs_likes) + "</likes>\n")
            f.write("<shares>" + str(refs_shares) + "</shares>\n")
            f.write(article_text)
            f.close()
        print ("page " + str(page_num) + " parsed")

            
ct_parser = news_coin_tel_parser()

for i in range(36,450):
    ct_parser.parse_page(i,verbose=False)

page 36 parsed
page 37 parsed
page 38 parsed
page 39 parsed
page 40 parsed
page 41 parsed
error in reading likes for  https://cointelegraph.com/explained/proof-of-work-explained
error reading text from page:  https://cointelegraph.com/explained/proof-of-work-explained
page 42 parsed
page 43 parsed
error in reading likes for  https://cointelegraph.com/explained/how-to-use-smart-contracts-for-revenue-sharing-explained
error reading text from page:  https://cointelegraph.com/explained/how-to-use-smart-contracts-for-revenue-sharing-explained
page 44 parsed
page 45 parsed
page 46 parsed
error in reading likes for  https://cointelegraph.com/explained/how-initial-bounty-offering-can-help-the-unbanked-explained
error reading text from page:  https://cointelegraph.com/explained/how-initial-bounty-offering-can-help-the-unbanked-explained
page 47 parsed
page 48 parsed
error in reading likes for  https://cointelegraph.com/explained/escrow-explained
error reading text from page:  https://cointelegr

page 125 parsed
page 126 parsed
page 127 parsed
error in reading likes for  https://cointelegraph.com/explained/uasf-vs-uahf-explained
error reading text from page:  https://cointelegraph.com/explained/uasf-vs-uahf-explained
page 128 parsed
page 129 parsed
page 130 parsed
error in reading likes for  https://cointelegraph.com/explained/bitcoin-and-blockchain-news-explained
error reading text from page:  https://cointelegraph.com/explained/bitcoin-and-blockchain-news-explained
page 131 parsed
page 132 parsed
page 133 parsed
page 134 parsed
page 135 parsed
page 136 parsed
page 137 parsed
error in reading likes for  https://cointelegraph.com/explained/bitcoin-price-explained
error reading text from page:  https://cointelegraph.com/explained/bitcoin-price-explained
error in reading likes for  https://cointelegraph.com/explained/what-are-cryptocurrency-debit-cards-explained
error reading text from page:  https://cointelegraph.com/explained/what-are-cryptocurrency-debit-cards-explained
page 1

AttributeError: 'list' object has no attribute 'keys'